In [1]:
from emg_fatigue.utils.load_emg_data import load_all_participant_data
from emg_fatigue.utils.process_emg_data import process_all_participant_data
from emg_fatigue.utils.create_loocv_dataset import create_loocv_dataset
from emg_fatigue.modeling.train import train_model
from emg_fatigue.modeling.evaluate import evaluate_model
from emg_fatigue.modeling.build.rnn_model import build_lstm_model
from emg_fatigue.plots.visualize_model_predictions import visualize_model_predictions
from emg_fatigue.config import BATCH_SIZE, PADDING_VALUE

2025-04-11 22:47:40.833266: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-11 22:47:40.834199: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-11 22:47:40.842291: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-11 22:47:40.857930: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744404460.887053 1167764 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744404460.89

In [2]:
raw_data = load_all_participant_data()
processed_data = process_all_participant_data(participant_data=raw_data)


# --- Create LOO-CV Datasets ---
all_participant_ids = list(processed_data.keys())
train_ids = all_participant_ids[:-3] 
val_ids = all_participant_ids[-3:-1]
test_ids = [all_participant_ids[-1]]

train_ds, val_ds, test_ds, input_shape, output_shape, norm_mean, norm_std = create_loocv_dataset(
    processed_data=processed_data,
    train_participant_ids=train_ids,
    validation_participant_ids=val_ids,
    test_participant_ids=test_ids,
    batch_size=BATCH_SIZE,
    padding_value=PADDING_VALUE,
    normalize=True,
    augment=True,
    time_param=10,
    time_masks=1,
    freq_param=10,
    freq_masks=1,
    augmentation_factor=1
)



2025-04-11 22:47:56.346 | INFO     | emg_fatigue.utils.create_loocv_dataset:create_loocv_dataset:162 - Processing data for datasets...
2025-04-11 22:47:56.347 | INFO     | emg_fatigue.utils.create_loocv_dataset:create_loocv_dataset:170 -   Processing train set (9 participants)...
2025-04-11 22:47:56.348 | INFO     | emg_fatigue.utils.create_loocv_dataset:create_loocv_dataset:170 -   Processing val set (2 participants)...
2025-04-11 22:47:56.348 | INFO     | emg_fatigue.utils.create_loocv_dataset:create_loocv_dataset:170 -   Processing test set (1 participants)...
2025-04-11 22:47:56.348 | INFO     | emg_fatigue.utils.create_loocv_dataset:create_loocv_dataset:233 - 
Found 513 frequency bins (features).
2025-04-11 22:47:56.348 | INFO     | emg_fatigue.utils.create_loocv_dataset:create_loocv_dataset:234 - Maximum sequence length across all sets: 125.
2025-04-11 22:47:56.348 | INFO     | emg_fatigue.utils.create_loocv_dataset:create_loocv_dataset:238 - Calculating normalization statistics 

2025-04-11 22:47:56.474180: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


2025-04-11 22:47:57.869 | INFO     | emg_fatigue.utils.create_loocv_dataset:create_loocv_dataset:294 -   Data augmentation complete. Training samples: 144 → 288
2025-04-11 22:47:57.870 | INFO     | emg_fatigue.utils.create_loocv_dataset:create_loocv_dataset:299 - Padding sequences...
2025-04-11 22:47:57.908 | INFO     | emg_fatigue.utils.create_loocv_dataset:create_loocv_dataset:333 -   Padded Train data shape: X=(288, 125, 513), y=(288, 125)
2025-04-11 22:47:57.914 | INFO     | emg_fatigue.utils.create_loocv_dataset:create_loocv_dataset:333 -   Padded Val data shape: X=(32, 125, 513), y=(32, 125)
2025-04-11 22:47:57.918 | INFO     | emg_fatigue.utils.create_loocv_dataset:create_loocv_dataset:333 -   Padded Test data shape: X=(16, 125, 513), y=(16, 125)
2025-04-11 22:47:57.918 | INFO     | emg_fatigue.utils.create_loocv_dataset:create_loocv_dataset:347 -   Standardized Training data shape: X=(288, 125, 513), y=(288, 125)
2025-04-11 22:47:57.918 | INFO     | emg_fatigue.utils.create_loo

In [3]:
model, model_name = build_lstm_model(input_shape=input_shape, test_id=test_ids, padding_value=PADDING_VALUE)

train_model(model=model, train_ds=train_ds, val_ds=val_ds, model_name=model_name, epochs=100)
evaluate_model(model=model, test_ds=test_ds, model_name=model_name)

2025-04-11 22:47:58.063 | INFO     | emg_fatigue.modeling.build.rnn_model:build_lstm_model:60 - Building model for test participant: P012
2025-04-11 22:47:58.348 | INFO     | emg_fatigue.modeling.train:train_model:48 - Model will be saved to: /home/lg519/Projects/Muscle_Fatigue_Research/emg_fatigue/models/lstm_P012.keras
2025-04-11 22:47:58.348 | INFO     | emg_fatigue.modeling.train:train_model:77 - Starting model training for up to 100 epochs...
Epoch 1/100


/home/lg519/.cache/pypoetry/virtualenvs/emg-fatigue-YbMeuwX9-py3.12/lib/python3.12/site-packages/keras/src/layers/layer.py:940: UserWarning: Layer 'squeeze_output' (of type Lambda) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
2025-04-11 22:48:12.278398: E tensorflow/core/util/util.cc:131] oneDNN supports DT_BOOL only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 665ms/step - loss: 2065.5613 - mae: 32.3196
Epoch 1: val_loss improved from inf to 1554.68982, saving model to /home/lg519/Projects/Muscle_Fatigue_Research/emg_fatigue/models/lstm_P012.keras
9/9 ━━━━━━━━━━━━━━━━━━━━ 24s 1s/step - loss: 2051.9143 - mae: 32.2657 - val_loss: 1554.6898 - val_mae: 30.9446 - learning_rate: 0.0010
Epoch 2/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 661ms/step - loss: 1405.7244 - mae: 29.7289
Epoch 2: val_loss improved from 1554.68982 to 1221.16504, saving model to /home/lg519/Projects/Muscle_Fatigue_Research/emg_fatigue/models/lstm_P012.keras
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 710ms/step - loss: 1399.7540 - mae: 29.7248 - val_loss: 1221.1650 - val_mae: 29.5659 - learning_rate: 0.0010
Epoch 3/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 682ms/step - loss: 1195.2689 - mae: 29.5949
Epoch 3: val_loss improved from 1221.16504 to 1130.54211, saving model to /home/lg519/Projects/Muscle_Fatigue_Research/emg_fatigue/models/lstm_P012.keras
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 726ms

{'loss': 245.84774780273438, 'mae': 8.881013870239258}

In [4]:
visualize_model_predictions(
    model=model,
    model_name=model_name,
    processed_data=processed_data,
    test_participant_ids=test_ids,
    input_shape=input_shape,
    norm_mean=norm_mean,
    norm_std=norm_std
)

2025-04-11 22:51:53.428 | INFO     | emg_fatigue.plots.visualize_model_predictions:visualize_model_predictions:53 - Starting prediction visualization for 1 test participants using model 'lstm_P012'.
2025-04-11 22:51:53.429 | INFO     | emg_fatigue.plots.visualize_model_predictions:visualize_model_predictions:57 - Normalization statistics provided. Applying normalization to test data.
2025-04-11 22:51:53.429 | INFO     | emg_fatigue.plots.visualize_model_predictions:visualize_model_predictions:71 - Visualizing predictions for participant P012...
2025-04-11 22:51:53.429 | INFO     | emg_fatigue.plots.visualize_model_predictions:visualize_model_predictions:85 -   Processing recording 1/8 for P012 left...
2025-04-11 22:51:53.430 | DEBUG    | emg_fatigue.plots.visualize_model_predictions:visualize_model_predictions:125 -   Applied normalization to spectrogram for P012 left Rec 1
2025-04-11 22:51:58.251 | INFO     | emg_fatigue.plots.visualize_model_predictions:visualize_model_predictions:27